<a href="https://colab.research.google.com/github/Daesung-Jung/baseball_pitchdesign/blob/main/execute_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 환경설정 확인
>  GPU 확인, 
깃허브 private clone 

In [ ]:
#코랩 GPU 확인
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

 # 추가 메모리 확인 
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!') 

In [ ]:
#import torch
#torch.cuda.empty_cache()

In [ ]:
!ssh-keygen -t rsa -b 4096 -C "shtnrgudsla@naver.com"

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

> 아래 나오는 키 복사해서 깃허브 계정 내 SSH~ 어쩌고 내용에 저장. title은 암거나.

In [ ]:
!cat /root/.ssh/id_rsa.pub

In [ ]:
!ssh -T git@github.com

!git config --global user.email "shtnrgudsla@naver.com"
!git config --global user.name "wjd890708!"

# 2. git clone 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#%cd /content/drive/MyDrive/
# 주소대신 ssh copy 
#!git clone git@github.com:Daesung-Jung/baseball_pitchdesign.git

In [ ]:
# 존재한다면 최신화
%cd /content/drive/MyDrive/baseball_pitchdesign/

#!git remote add origin /content/drive/MyDrive/baseball_pitchdesign/
!git reset --hard HEAD
!git pull

#!git pull 
#필요한 패키지 설치
%cd /content/drive/MyDrive/baseball_pitchdesign/
!pip install -r requirements.txt

In [ ]:
import yaml
%cd /
from glob import glob

img_list = glob('/content/drive/MyDrive/baseball_pitchdesign/images/*.jpg')
lab_list = glob('/content/drive/MyDrive/baseball_pitchdesign/labels/*.txt')

print(len(img_list))
print(len(lab_list))

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

In [ ]:
with open('/content/drive/MyDrive/baseball_pitchdesign/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/baseball_pitchdesign/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

with open('/content/drive/MyDrive/baseball_pitchdesign/data.yaml', 'r') as f:
  data = yaml.load(f)

print(data)
data['train'] = '/content/drive/MyDrive/baseball_pitchdesign/train.txt'
data['val'] = '/content/drive/MyDrive/baseball_pitchdesign/val.txt'

print(data['val'])
with open('/content/drive/MyDrive/baseball_pitchdesign/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

# 3. 학습하기 

In [ ]:
%cd /content/drive/MyDrive/baseball_pitchdesign/yolov5/
!python train.py --img 416 --batch 32 --epochs 600 --data /content/drive/MyDrive/baseball_pitchdesign/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name elbow_bg_all_32_600

# 4. 결과 비교

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/baseball_pitchdesign/yolov5/runs/

In [ ]:
file_list_[144]


# 5. 실제 데이터 넣기

> 원하는 만큼 input, output 숫자로 넣기


In [ ]:
import os

%cd /content/drive/MyDrive/baseball_pitchdesign/yolov5/


def cap_slomo(path = "/content/drive/MyDrive/project_ddg/rawdata_video/2/"):
       



  path = path
  file_list_ = os.listdir(path)
  file_list_ = sorted(file_list_)
  print("슬로우모션을 뽑아낼 데이터, [1~",len(file_list_),"번째] 중"
  "\n처음(start)과 마지막(last)를 <숫자만> 입력하세요" )
  start = int(input("처음 : "))
  #print("처음 : ",start)
  end = int(input("마지막 : "))


  for aa in range(start-1,end):

    print(aa+1, file_list_[aa], "실행합니다")
    #변수명을 직접 넣으면 안돼서 {}활용 
    file_name_=file_list_[aa]
    !python detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source '{path}{file_name_}' --nosave
    
#괄호안에 해당 디렉토리를 넣어주세요. 디폴트는 2로 설정!

#가령 윤수는 cap_slomo("/content/drive/MyDrive/rawdata_video/3/")
#안 넣으면 디폴트

cap_slomo()




In [ ]:
import os

%cd /content/drive/MyDrive/baseball_pitchdesign/yolov5/


def cap_slomo(path = "/content/drive/MyDrive/project_ddg/rawdata_video/2/"):
       
  path = path
  file_list_ = os.listdir(path)
  file_list_ = sorted(file_list_)
  print("슬로우모션을 뽑아낼 데이터, [1~",len(file_list_),"번째] 중"
  "\n처음(start)과 마지막(last)를 <숫자만> 입력하세요" )
  start = int(input("처음 : "))
  #print("처음 : ",start)
  end = int(input("마지막 : "))


  for aa in range(start-1,end):

    print(aa+1, file_list_[aa], "실행합니다")
    #변수명을 직접 넣으면 안돼서 {}활용 
    file_name_=file_list_[aa]
    !python detect_re.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source '{path}{file_name_}' --nosave
    
#괄호안에 해당 디렉토리를 넣어주세요. 디폴트는 2로 설정!

#가령 윤수는 cap_slomo("/content/drive/MyDrive/rawdata_video/3/")
#안 넣으면 디폴트

cap_slomo()




In [ ]:
#수동으로 작업 

%cd /content/drive/MyDrive/baseball_pitchdesign/yolov5/
!python detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.8 --source /content/drive/MyDrive/dataset/20210414KTOB02021-2.mp4 --nosave

In [ ]:
pip uninstall lightning-flash

In [ ]:
#!git clone https://github.com/PyTorchLightning/lightning-flash.git
!pip install pytorch-lightning
!pip install lightning-flash
!pip install kornia
!pip install pytorchvideo
#!pip install torch

In [ ]:
import os
import sys
sys.path.append("/content/drive/MyDrive/lightning-flash")

In [ ]:
import os
from typing import Callable, List
import pytorch_lightning
import pytorchvideo.data
import torch.utils.data
from pytorchvideo.data.encoded_video import EncodedVideo

import kornia.augmentation as K
import torch
from pytorchvideo.transforms import ApplyTransformToKey, RandomShortSideScale, UniformTemporalSubsample
from torch.utils.data.sampler import RandomSampler
from torchvision.transforms import CenterCrop, Compose, RandomCrop, RandomHorizontalFlip

import flash
from flash.core.finetuning import NoFreeze
from flash.core.classification import Labels
from flash.video import VideoClassificationData, VideoClassifier

from flash.data.utils import download_data

In [ ]:

download_data("https://pl-flash-data.s3.amazonaws.com/kinetics.zip")

In [ ]:
# 2. [Optional] Specify transforms to be used during training.
post_tensor_transform = [UniformTemporalSubsample(8), RandomShortSideScale(min_size=256, max_size=320)]

per_batch_transform_on_device = [K.Normalize(torch.tensor([0.45, 0.45, 0.45]), torch.tensor([0.225, 0.225, 0.225]))]

train_post_tensor_transform = post_tensor_transform + [RandomCrop(244), RandomHorizontalFlip(p=0.5)]

val_post_tensor_transform = post_tensor_transform + [CenterCrop(244)]
train_per_batch_transform_on_device = per_batch_transform_on_device


def make_transform(
    post_tensor_transform: List[Callable] = post_tensor_transform,
    per_batch_transform_on_device: List[Callable] = per_batch_transform_on_device
):
    return {
        "post_tensor_transform": Compose([
            ApplyTransformToKey(
                key="video",
                transform=Compose(post_tensor_transform),
            ),
        ]),
        "per_batch_transform_on_device": Compose([
            ApplyTransformToKey(
                key="video",
                transform=K.VideoSequential(
                  *per_batch_transform_on_device, data_format="BCTHW", same_on_frame=False)
            ),
        ]),
    }

In [ ]:

# 2. Load the data
datamodule = VideoClassificationData.from_folders(
    train_folder="/content/drive/MyDrive/data/train/",
    val_folder="/content/drive/MyDrive/data/val/",
    test_folder="/content/drive/MyDrive/data/predict/",
    train_transform=make_transform(train_post_tensor_transform),
    val_transform=make_transform(val_post_tensor_transform),
    predict_transform=make_transform(val_post_tensor_transform),
    batch_size=8,
    clip_sampler="uniform",
    clip_duration=2,
    video_sampler="random",
    decode_audio=False,
)

#6. GCP Storage to Colab


In [ ]:
#접근, 인증필요함 
from google.colab import auth
auth.authenticate_user()

project_id = 'dadangae'
!gcloud config set project dadangae


In [ ]:
!gcloud auth login

In [ ]:
!gcloud auth list

In [ ]:
!gcloud config set project dadangae

In [ ]:
#실행
!gcloud init

In [ ]:
2
!gsutil ls

In [ ]:
1%cd ..

In [ ]:
#gcp to google drive 
#-m 붙이면 가속화할 수 있으나 파일명 확장자가 조금 이상하게 떨어짐 
!gsutil cp -r gs://diego/2/* /content/drive/MyDrive/project_ddg/rawdata_video/2


#7. 쉘 언어에서 
1. 해당 파일의 리스트를 받기
2. 쉘에서 변수명을 실행할 땐 앞엔 $를 붙여야 함  -> 일종의 이스케이프(\) 개념


In [ ]:

'''
$ gsutil list                           # 나의 버킷 리스트 보기
$ gsutil ls -r gs://버킷이름             # 버킷 안에 들어있는 파일 확인
$ gsutil du -s gs://버킷이름             # 버킷 용량 확인
$ gsutil mb gs://버킷이름                # 버킷 생성
$ gsutil rb gs://버킷이름                # 버킷 삭제
$ gsutil cp 로컬 파일 위치 gs://버킷이름   # 로컬 -> 버킷 복사
$ gsutil cp gs://버킷이름 로컬 파일 위치   # 버킷 -> 로컬 복사
$ gsutil mv 로컬 파일 위치 gs://버킷이름   # 로컬 -> 버킷 이동
$ gsutil mv gs://버킷이름 로컬 파일 위치   # 버킷 -> 로컬 이동
$ gsutil rm gs://버킷이름/파일이름        # 파일 삭제
$ gsutil ls -L gs://버킷이름/파일이름     # 파일 정보 보기
'''

In [ ]:
#GCP 해당 폴더 리스트 만들기 
df = !gsutil ls -r gs://diego/2
print(df)

In [ ]:
#최상위에 temp_video 폴더를  생성
%cd \
%cd ..
%mkdir temp_video

In [ ]:
!pip install firebase_admin

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage
import datetime

import urllib.request as req
import cv2

cred = credentials.Certificate('/content/adc.json')
app = firebase_admin.initialize_app(cred, {'storageBucket': 'cnc-designs.appspot.com'}, name='storage')
bucket = storage.bucket(app=app)

def generate_image_url(blob_path):
    """ generate signed URL of a video stored on google storage. 
        Valid for 300 seconds in this case. You can increase this 
        time as per your requirement. 
    """                                                        
    blob = bucket.blob(blob_path) 
    return blob.generate_signed_url(datetime.timedelta(seconds=300), method='GET')


url = generate_image_url('sample1.mp4')
req.urlretrieve(url, "sample1.mp4")
cap = cv2.VideoCapture('sample1.mp4')

if cap.isOpened():
    print ("File Can be Opened")
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()
        #print cap.isOpened(), ret
        if frame is not None:
            # Display the resulting frame
            cv2.imshow('frame',frame)
            # Press q to close the video windows before it ends if you want
            if cv2.waitKey(22) & 0xFF == ord('q'):
                break
        else:
            print("Frame is None")
            break
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    print ("Video stop")
else:
    print("Not Working")

In [ ]:

%cd  temp_video
for aa in range(6,12)):
  q=df[aa]
  print(q, "다운로드 중")
  !gsutil cp $q .
  

In [ ]:
!gsutil cp $q .

In [ ]:
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source /content/drive/MyDrive/rawdata_video/2/20210414LGWO02021-3.mp4 --nosave

In [ ]:
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source !gsutil cp $!gsutil cp $q --nosave

In [ ]:
#구글 클라우드 SDK 설치 
%cd /content/drive/MyDrive/
!curl https://sdk.cloud.google.com | bash

In [ ]:
#Download file from Cloud Storage to Google Colab
#!gsutil cp gs://diego/2/20210414KTOB02021-1.mp4 .
!gsutil cp gs://diego/2/20210414KTOB02021-2.mp4 .
!gsutil cp gs://diego/2/20210414KTOB02021-3.mp4 .
!gsutil cp gs://diego/2/20210414KTOB02021-4.mp4 .
!gsutil cp gs://diego/2/20210414LGWO02021-1.mp4 .
#24초 걸림 (2.6기가)


#Upload file from Google Colab to Cloud
#!gsutil cp 20210624NCLT02021-1.mp4 gs://diego/example/

In [ ]:
#GCP에서 구글 드라이브로 (45초)
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-1.mp4
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-2.mp4
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-3.mp4
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-4.mp4

In [ ]:
#실제 진행 
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source 20210414LGWO02021-2.mp4 --nosave
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source 20210414LGWO02021-3.mp4 --nosave
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source 20210414LGWO02021-4.mp4 --nosave
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source 20210414LTHT02021-1.mp4 --nosave

!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source 20210414LTHT02021-2.mp4 --nosave
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source 20210414LTHT02021-3.mp4 --nosave



In [ ]:
!pip install av
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar


In [ ]:
pip install git+https://github.com/Atze00/MoViNet-pytorch.git


In [ ]:
# Extract and organize video data..
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
import os
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar


#8. 구종 동영상 폴더 자동 분류(이동) 및 전처리

In [ ]:
#우리가 취합한 데이터 파일 
import pandas as pd
data_list = pd.read_csv("/content/drive/MyDrive/project_ddg/total__.csv")
data_list.head()
data_list=data_list[['pitcher','pitch_type','gameId','cumsum','no']]
data_list.groupby('pitch_type').agg('count')['pitcher']
data_list['file_name']= data_list['gameId']+"_"+data_list['no'].astype(str)+".mp4"
data_list['pitch_type'] = data_list['pitch_type'].replace("슬라이더","SL").replace("포크","FO").replace("직구","FF").replace("커터","FC").replace("투심","FT").replace("체인지업","CH").replace("커브","CU")
data_list['pitch_type'].unique()

In [ ]:
'''
#동영상 데이터 복사해서 Class 별로 이동 
import os
main="/content/drive/MyDrive/project_ddg/"
for (path, directory, files) in os.walk("/content/drive/MyDrive/project_ddg/zoom_in/videos"):
  for aa in files:   
    exist_ = len(data_list[data_list['file_name']==aa])
    print(aa)
    if exist_!=0:      
      print("-------------------------------------"+aa+"-------------------------------------")
      pitch_type = data_list[data_list['file_name']==aa]['pitch_type']
      pitch_type = pitch_type.values[0]
      !cp $path"/"$aa $main"class_video/"$pitch_type"/"$aa

%cd /content/drive/MyDrive/project_ddg/train_test_split
#해당 폴더 내 폴더 생성 
!mkdir CH
!mkdir CU
!mkdir FC
!mkdir FF
!mkdir FO
!mkdir FT
!mkdir SL

import os
from sklearn.model_selection import train_test_split

def split_folder(folder_name):
  X= os.listdir("/content/drive/MyDrive/project_ddg/class_video/"+str(folder_name))
  #7:3 스플릿, 섞기
  X_train, X_test = train_test_split(X, test_size=0.3, shuffle=True, random_state=1004)
  #파일 이름, 한 줄씩 처리
  with open("/content/drive/MyDrive/project_ddg/train_test_split/{0}1.txt".format(str(folder_name)), "w") as file:
    for aa in X_train:
      file.write("{0} 1\n".format(str(aa)))
    for bb in X_test:
      file.write("{0} 2\n".format(str(bb)))  

split_folder("FF")
split_folder("FC")
split_folder("FO")
split_folder("FT")
split_folder("SL")
split_folder("CU")
split_folder("CH")


#동영상 데이터 복사해서 Class 별로 이동 
import os
main="/content/drive/MyDrive/project_ddg/"
for (path, directory, files) in os.walk("project_ddg"):
  for aa in files:   
    exist_ = len(data_list[data_list['file_name']==aa])
    print(aa)
    if exist_!=0:      
      print("-------------------------------------"+aa+"-------------------------------------")
      pitch_type = data_list[data_list['file_name']==aa]['pitch_type']
      pitch_type = pitch_type.values[0]
      !cp $path"/"$aa $main"class_video/"$pitch_type"/"$aa

%cd /content/drive/MyDrive/project_ddg
#해당 폴더 내 폴더 생성 
!mkdir sort_video

#클래스 비디오 안에 있는 구종_파일이름을 sor_vidoe로 
import os
main="/content/drive/MyDrive/project_ddg"
for (path, directory, files) in os.walk(str(main)+"/class_video"):
  for aa in files:    
    new=path.split("/")[-1]+"_"+str(aa)
    !cp $path"/"$aa $main"/sort_video/"$new 
'''

%cd /content/drive/MyDrive/project_ddg
#해당 폴더 내 폴더 생성 
!mkdir videos

#동영상 데이터 복사해서 Class 별로 이동 
import os
main="/content/drive/MyDrive/project_ddg/videos"
for (path, directory, files) in os.walk("/content/drive/MyDrive/project_ddg/zoom_in/videos"):
  for aa in files:   
    exist_ = len(data_list[data_list['file_name']==aa])
    print(aa)
    if exist_!=0:      
      print("-------------------------------------"+aa+"-------------------------------------")
      pitch_type = data_list[data_list['file_name']==aa]['pitch_type']
      pitch_type = pitch_type.values[0]
      !cp $main"/"$aa $main"/data/"$pitch_type"_"$aa

In [ ]:
#구종 구분
import pandas as pd
files = os.listdir( "/content/drive/MyDrive/project_ddg/videos/")
files = pd.DataFrame(files)
files['pitch_type'] = pd.DataFrame(files)[0].str[0:2]

#트레인 테스트 나누기
for aa in files['pitch_type'].unique():
  X =files[files['pitch_type']==aa]
  X_train, X_test = train_test_split(X, test_size=0.3, shuffle=True, random_state=1004)
  files.loc[files[0].isin(X_train[0]),'type'] = 'train'
  files.loc[files[0].isin(X_test[0]),'type'] = 'test'
files.columns= ["video_id","label","type"]
real_file=files

%cd /content/drive/MyDrive/project_ddg/videos/
#해당 폴더 내 폴더 생성 
!mkdir train
!mkdir test

import os
main="/content/drive/MyDrive/project_ddg/videos"
for (path, directory, files) in os.walk(str(main)):
  for cc in files:    
    #파일이름 
    folder = real_file[real_file['video_id']==cc]['type'].values[0]
    #train test 갈라서 해당 폴더에 넣기
    !mv $path"/"$cc "/content/drive/MyDrive/project_ddg/videos/"$folder"/"$cc 


train_list = files[files['type']=="train"]
test_list = files[files['type']=="test"]

del train_list['type']
train_list.to_csv("/content/drive/MyDrive/project_ddg/videos/train/train_labels.csv",index=False)

del test_list['type']
test_list.to_csv("/content/drive/MyDrive/project_ddg/videos/test/test_labels.csv",index=False)



In [ ]:
#zip -r ~ ~ 압축하는 데 뒷 경로가 상위 디렉토리가 포함되어 있음 이거 어캐함
#이후 삭제 


#9.Data DownLoad


In [ ]:
!pip install -q tf-models-nightly tfds-nightly
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#%cd /content/drive/MyDrive/
# 주소대신 ssh copy 
#!git clone git@github.com:Daesung-Jung/baseball_pitchdesign.git

In [ ]:
#!tfds new my_dataset
!rm -rf /root/tensorflow_datasets/
%cd /content/drive/MyDrive/project_ddg/
!tfds build my_dataset

In [ ]:
import my_dataset.my_dataset as my_ds
import tensorflow_datasets as tfds
builder = my_ds.MyDataset()
config = tfds.download.DownloadConfig(verify_ssl=False)
builder.download_and_prepare(download_config=config)

#9. VideoClassification on PyTorch!
> 구종 자동분류!

In [ ]:
import os
from six.moves import urllib
import matplotlib.pyplot as plt
import mediapy as media
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from official.vision.beta.configs import video_classification
from official.vision.beta.projects.movinet.configs import movinet as movinet_configs
from official.vision.beta.projects.movinet.modeling import movinet
from official.vision.beta.projects.movinet.modeling import movinet_layers
from official.vision.beta.projects.movinet.modeling import movinet_model

In [ ]:
movinet_a2_hub_url = 'https://tfhub.dev/tensorflow/movinet/a2/base/kinetics-600/classification/1'

inputs = tf.keras.layers.Input(
    shape=[None, None, None, 3],
    dtype=tf.float32)

encoder = hub.KerasLayer(movinet_a2_hub_url, trainable=True)

# Important: To use tf.nn.conv3d on CPU, we must compile with tf.function.
encoder.call = tf.function(encoder.call, experimental_compile=True)

# [batch_size, 600]
outputs = encoder(dict(image=inputs))

model = tf.keras.Model(inputs, outputs)

In [ ]:
num_classes = builder.info.features['label'].num_classes
num_examples = {
    name: split.num_examples
    for name, split in builder.info.splits.items()
}

print('Number of classes:', num_classes)
print('Number of examples for train:', num_examples['train'])
print('Number of examples for test:', num_examples['test'])
print()

builder.info

In [ ]:
batch_size = 8
num_frames = 8
frame_stride = 10
resolution = 172

def format_features(features):
  video = features['video']
  video = video[:, ::frame_stride]
  video = video[:, :num_frames]

  video = tf.reshape(video, [-1, video.shape[2], video.shape[3], 3])
  video = tf.image.resize(video, (resolution, resolution))
  video = tf.reshape(video, [-1, num_frames, resolution, resolution, 3])
  video = tf.cast(video, tf.float32) / 255.

  label = tf.one_hot(features['label'], num_classes)
  return (video, label)

train_dataset = builder.as_dataset(
    split='train',
    batch_size=batch_size,
    shuffle_files=True)
train_dataset = train_dataset.map(
    format_features,
    num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.prefetch(2)

test_dataset = builder.as_dataset(
    split='test',
    batch_size=batch_size)
test_dataset = test_dataset.map(
    format_features,
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=True)
test_dataset = test_dataset.prefetch(2)

In [ ]:
videos, labels = next(iter(train_dataset))
media.show_videos(videos.numpy(), codec='gif', fps=5)

In [ ]:
videos.shape

In [ ]:
labels

In [ ]:
train_dataset

In [ ]:
model_id = 'a0'

tf.keras.backend.clear_session()

backbone = movinet.Movinet(
    model_id=model_id)
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=7)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Load pretrained weights from TF Hub
movinet_hub_url = f'https://tfhub.dev/tensorflow/movinet/{model_id}/base/kinetics-600/classification/1'
movinet_hub_model = hub.KerasLayer(movinet_hub_url, trainable=True)
pretrained_weights = {w.name: w for w in movinet_hub_model.weights}
model_weights = {w.name: w for w in model.weights}
for name in pretrained_weights:
  model_weights[name].assign(pretrained_weights[name])

# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=num_classes)
model.build([batch_size, num_frames, resolution, resolution, 3])

# Freeze all layers except for the final classifier head
for layer in model.layers[:-1]:
  layer.trainable = False
model.layers[-1].trainable = True

In [ ]:
num_epochs = 3

train_steps = num_examples['train'] // batch_size
total_train_steps = train_steps * num_epochs
test_steps = num_examples['test'] // batch_size

loss_obj = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1)

metrics = [
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=1, name='top_1', dtype=tf.float32),
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=5, name='top_5', dtype=tf.float32),
]

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)
optimizer = tf.keras.optimizers.RMSprop(
    learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

callbacks = [
    tf.keras.callbacks.TensorBoard(),
]

In [ ]:
results = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=num_epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks,
    validation_freq=1,
    verbose=1)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs --port 0

#Video Classification with Transformers


In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
!wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
!tar xf ucf101_top5.tar.gz

In [ ]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 5

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")


In [ ]:

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)


def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = cropped.numpy().squeeze()
    return cropped


# Following method is modified from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def load_video(path, max_frames=0):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center(frame)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


# Label preprocessing with StringLookup.
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)
print(label_processor.get_vocabulary())


def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))

        # Pad shorter videos.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate(frames, padding)

        frames = frames[None, ...]

        # Initialize placeholder to store the features of the current video.
        temp_frame_featutes = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[1]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_featutes[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_featutes[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_featutes.squeeze()

    return frame_features, labels

In [ ]:
!wget -q https://git.io/JZmf4 -O top5_data_prepared.tar.gz
!tar xf top5_data_prepared.tar.gz

In [ ]:
train_data, train_labels = np.load("train_data.npy"), np.load("train_labels.npy")
test_data, test_labels = np.load("test_data.npy"), np.load("test_labels.npy")

print(f"Frame features in train set: {train_data.shape}")

In [ ]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    classes = len(label_processor.get_vocabulary())

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = "/content/drive/MyDrive/project_ddg/class_video/CH"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        validation_split=0.15,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

In [ ]:
trained_model = run_experiment()

In [ ]:
import os
import sys
from typing import Callable, List

import torch
from torch.utils.data.sampler import RandomSampler

import flash
from flash.core.classification import Labels
from flash.core.data.utils import download_data
from flash.core.finetuning import NoFreeze
from flash.core.utilities.imports import _KORNIA_AVAILABLE, _PYTORCHVIDEO_AVAILABLE
from flash.video import VideoClassificationData, VideoClassifier

if _PYTORCHVIDEO_AVAILABLE and _KORNIA_AVAILABLE:
    import kornia.augmentation as K
    from pytorchvideo.transforms import ApplyTransformToKey, RandomShortSideScale, UniformTemporalSubsample
    from torchvision.transforms import CenterCrop, Compose, RandomCrop, RandomHorizontalFlip
else:
    print("Please, run `pip install torchvideo kornia`")
    sys.exit(1)

if __name__ == '__main__':

    # 1. Download a video clip dataset. Find more dataset at https://pytorchvideo.readthedocs.io/en/latest/data.html
 #   download_data("https://pl-flash-data.s3.amazonaws.com/kinetics.zip")

    # 2. [Optional] Specify transforms to be used during training.
    # Flash helps you to place your transform exactly where you want.
    # Learn more at:
    # https://lightning-flash.readthedocs.io/en/latest/general/data.html#flash.core.data.process.Preprocess
    post_tensor_transform = [UniformTemporalSubsample(8), RandomShortSideScale(min_size=256, max_size=320)]
    per_batch_transform_on_device = [K.Normalize(torch.tensor([0.45, 0.45, 0.45]), torch.tensor([0.225, 0.225, 0.225]))]

    train_post_tensor_transform = post_tensor_transform + [RandomCrop(244), RandomHorizontalFlip(p=0.5)]
    val_post_tensor_transform = post_tensor_transform + [CenterCrop(244)]
    train_per_batch_transform_on_device = per_batch_transform_on_device

    def make_transform(
        post_tensor_transform: List[Callable] = post_tensor_transform,
        per_batch_transform_on_device: List[Callable] = per_batch_transform_on_device
    ):
        return {
            "post_tensor_transform": Compose([
                ApplyTransformToKey(
                    key="video",
                    transform=Compose(post_tensor_transform),
                ),
            ]),
            "per_batch_transform_on_device": Compose([
                ApplyTransformToKey(
                    key="video",
                    transform=K.VideoSequential(
                        *per_batch_transform_on_device, data_format="BCTHW", same_on_frame=False
                    )
                ),
            ]),
        }

    # 3. Load the data from directories.
    datamodule = VideoClassificationData.from_folders(
        train_folder=os.path.join(flash.PROJECT_ROOT, "data/kinetics/train"),
        val_folder=os.path.join(flash.PROJECT_ROOT, "data/kinetics/val"),
        predict_folder=os.path.join(flash.PROJECT_ROOT, "data/kinetics/predict"),
        train_transform=make_transform(train_post_tensor_transform),
        val_transform=make_transform(val_post_tensor_transform),
        predict_transform=make_transform(val_post_tensor_transform),
        batch_size=8,
        clip_sampler="uniform",
        clip_duration=1,
        video_sampler=RandomSampler,
        decode_audio=False,
        num_workers=8
    )

    # 4. List the available models
    print(VideoClassifier.available_backbones())
    # out: ['efficient_x3d_s', 'efficient_x3d_xs', ... ,slowfast_r50', 'x3d_m', 'x3d_s', 'x3d_xs']
    print(VideoClassifier.get_backbone_details("x3d_xs"))

    # 5. Build the VideoClassifier with a PyTorchVideo backbone.
    model = VideoClassifier(
        backbone="x3d_xs", num_classes=datamodule.num_classes, serializer=Labels(), pretrained=False
    )

    # 6. Finetune the model
    trainer = flash.Trainer(fast_dev_run=True)
    trainer.finetune(model, datamodule=datamodule, strategy=NoFreeze())

    trainer.save_checkpoint("video_classification.pt")

    # 7. Make a prediction
    predictions = model.predict(os.path.join(flash.PROJECT_ROOT, "data/kinetics/predict"))
    print(predictions)

In [ ]:
%cd /content/drive/MyDrive
#quiet 하위 디렉토리만 

!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/drive/MyDrive/models/official/vision/beta/projects/movinet
!pip install -r requirements.txt

In [ ]:
import tensorflow as tf

from official.vision.beta.projects.movinet.modeling import movinet
from official.vision.beta.projects.movinet.modeling import movinet_model

# Create backbone and model.
backbone = movinet.Movinet(
    model_id='a0',
    causal=True,
    use_external_states=True,
)
model = movinet_model.MovinetClassifier(
    backbone, num_classes=600, output_states=True)

# Create your example input here.
# Refer to the paper for recommended input shapes.
inputs = tf.ones([1, 8, 172, 172, 3])

# [Optional] Build the model and load a pretrained checkpoint
model.build(inputs.shape)

checkpoint_dir = '/path/to/checkpoint'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

# Run the model prediction.
output = model(inputs)
prediction = tf.argmax(output, -1)

In [ ]:
import tensorflow as tf

from official.vision.beta.projects.movinet.modeling import movinet
from official.vision.beta.projects.movinet.modeling import movinet_model

# Create backbone and model.
backbone = movinet.Movinet(
    model_id='a0',
    causal=True,
    use_external_states=True,
)
model = movinet_model.MovinetClassifier(
    backbone, num_classes=7, output_states=True)

# Create your example input here.
# Refer to the paper for recommended input shapes.
inputs = tf.ones([1, 8, 172, 172, 3])

# [Optional] Build the model and load a pretrained checkpoint
model.build(inputs.shape)

checkpoint_dir = '/path/to/checkpoint'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
status.assert_existing_objects_matched()

# Run the model prediction.
output = model(inputs)
prediction = tf.argmax(output, -1)

In [ ]:
!pip install av
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

In [ ]:
pip install git+https://github.com/Atze00/MoViNet-pytorch.git


In [ ]:
# Extract and organize video data..
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
import os
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar


In [ ]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [ ]:
# open the .txt file which have names of training videos
f = open("trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()